In [4]:
import pandas as pd
import json
import os
import operator
from pandas import json_normalize
import pandasql as psql
import numpy as np

In [98]:
#! pip install pandasql

# weather


In [5]:
ress=[]
dir_lis= os.listdir('/Users/shwethailango/Documents/Work/Solarillion/Flight_delay_pred/Data/weather')
for z in dir_lis:
    res=[]
    aa='/Users/shwethailango/Documents/Work/Solarillion/Flight_delay_pred/Data/weather/'+z
    file_lis= os.listdir(aa)


    for file in file_lis:
        
        if (operator.contains(file, "2016")) | (operator.contains(file, "2017")):
            
            


            
            path=aa+'/'+file
            with open(path) as f:
                d = json.load(f)

            df1 = pd.json_normalize(d['data']['weather'])
            df3=df1[['hourly','date']]

            li=[]
            air=d['data']['request'][0]['query'][:3] #airport name with 1st 3 letters
            for i in range(0,len(df3)):
                k=df3.iloc[i,0] # weather details "0" for all 31 dates
                k1=df3.iloc[i,1] # date "1" --all 31 dates
                cols=['weatherCode' ,'pressure', 'cloudcover', 'tempF', 'WindChillF' ,'time',  'precipMM', 'DewPointF', 'humidity', 'windspeedKmph', 'visibility', 'WindGustKmph','winddirDegree']
                len(cols)
                df4=pd.DataFrame(k)
                df4=df4[cols]
                df4['date']=k1      #0-23 hrs of a day
                df4['airport']=air    
                li.append(df4)   #append all 31 days in a month(1 file)

            r=pd.concat(li,ignore_index=True)  #23hrs * 31days
            res.append(r)  

    rr=pd.concat(res,ignore_index=True) # append 24 files(24 months)
    ress.append(rr)                     
    
result_w=pd.concat(ress,ignore_index=True) # for 15 airports

In [6]:
print(df4.shape,
r.shape,
rr.shape,
result_w.shape)

(24, 15) (744, 15) (17544, 15) (263160, 15)


In [7]:
print(len(li),
len(res),
len(ress))

31 24 15


In [9]:
result_w=result_w[['airport','date','time','weatherCode', 'pressure', 'cloudcover', 'tempF', 'WindChillF',
         'precipMM', 'DewPointF', 'humidity', 'windspeedKmph',
        'visibility', 'WindGustKmph', 'winddirDegree']]
result_w.to_csv('/Users/shwethailango/Documents/Work/Solarillion/Flight_delay_pred/Data/data_processed/weather.csv')

# flight

In [11]:
lis=[]
for root, dirs, files in os.walk('/Users/shwethailango/Documents/Work/Solarillion/Flight_delay_pred/Data/flight'):
    for i in files:
        if i.endswith('.csv'):
            lis.append(os.path.join(root,i))
len(lis)      

li2=[]           
for j in lis:
        d1=pd.read_csv(j,low_memory=False)
        ap=['ATL','CLT','DEN','DFW','EWR','IAH','MCO','ORD','SEA','JFK','LAS','LAX','MIA','PHX','SFO']
        d2=d1[(d1.Origin.isin(ap))] #do not filter at a time...filter origin nd then filter dest frm it!(or vic-ves)
        d2=d2[(d2.Dest.isin(ap))]
        col=['FlightDate','Origin','CRSDepTime', 'CRSArrTime', 'DayofMonth', 'DepDelayMinutes', 'Quarter' ,'Year','DepTime', 'DepDel15', 'OriginAirportID', 'DestAirportID', 'ArrDel15', 'ArrDelayMinutes','Month' , 'ArrTime']
        d3=d2[col]
        li2.append(d3)   

r2=pd.concat(li2,ignore_index=True)

In [12]:
r2.to_csv('/Users/shwethailango/Documents/Work/Solarillion/Flight_delay_pred/Data/data_processed/flight.csv')

# merge

In [13]:
import math
v=0
def ceil_(number):
        
        if (number % 100) < 30:
            c=math.ceil(number-(number%100)) 
        else:
            c=math.ceil(number/100)*100
            
        if(c>2300):
            return v
        else:
            return c
    

In [14]:
#rounding CRSdeptime to nearest hundred (till 30), so that it can be matched with 'time' in weather data.For eg:,
print(ceil_(2310),ceil_(2350),ceil_(30),ceil_(50))

2300 0 100 100


In [15]:
r2['CRSDepTime1']=[ceil_(i) for i in r2.CRSDepTime]


In [16]:
result_w.time=result_w.time.astype(np.int64)

In [17]:
final= pd.merge(r2,result_w, left_on = ['FlightDate', 'Origin', 'CRSDepTime1'], right_on = ['date', 'airport', 'time']) 

In [18]:
final.shape
#final.columns

(1877296, 32)

# Grooming Data

In [19]:
data=final.dropna()

In [22]:
data.shape

(1851436, 32)

In [24]:
#data=data.drop(columns=['Origin','CRSDepTime1','Unnamed: 0_x','Unnamed: 0_y'] , axis=1)
data.columns

Index(['FlightDate', 'Origin', 'CRSDepTime', 'CRSArrTime', 'DayofMonth',
       'DepDelayMinutes', 'Quarter', 'Year', 'DepTime', 'DepDel15',
       'OriginAirportID', 'DestAirportID', 'ArrDel15', 'ArrDelayMinutes',
       'Month', 'ArrTime', 'CRSDepTime1', 'airport', 'date', 'time',
       'weatherCode', 'pressure', 'cloudcover', 'tempF', 'WindChillF',
       'precipMM', 'DewPointF', 'humidity', 'windspeedKmph', 'visibility',
       'WindGustKmph', 'winddirDegree'],
      dtype='object')

In [25]:
data=data[['FlightDate', 'airport','time','DayofMonth','Quarter','Month', 'Year', 'DayofMonth','OriginAirportID','CRSDepTime',
         'DepTime','DepDelayMinutes','DepDel15','DestAirportID','CRSArrTime','ArrTime','ArrDel15', 'ArrDelayMinutes','weatherCode', 'pressure', 'cloudcover', 'tempF', 'WindChillF',
       'precipMM', 'DewPointF', 'humidity', 'windspeedKmph', 'visibility',
       'WindGustKmph', 'winddirDegree']]

In [27]:
data.to_csv('/Users/shwethailango/Documents/Work/Solarillion/Flight_delay_pred/Data/data_processed/flight_delay_pred.csv')

#                                                   -------

In [28]:
result_w.shape #weather_data
r2.shape       #flight_data
data.shape     #final_data


(1851436, 30)